# или так (исправлен момент с h)

import numpy as np

def solve_system(matr, vec, n):
    # n — это размер системы (количество узлов)

    # Массивы для решения методом прогонки
    res = np.zeros(n)
    mu = np.zeros(n)
    nu = np.zeros(n)

    # Прямой ход прогонки (прямой проход)
    # Учитываем, что matr[:, 0] - поддиагональ, matr[:, 1] - главная диагональ, matr[:, 2] - наддиагональ
    mu[0] = vec[0] / matr[0][0]  # делим на первый элемент главной диагонали
    nu[0] = -matr[0][1] / matr[0][0]  # - matr[0][2] (если бы было три диагонали)

    # Проходим по всем элементам от 1 до n-1
    for i in range(1, n - 1):
        denominator = matr[i][i] + matr[i][i - 1] * nu[i - 1]
        mu[i] = (vec[i] - matr[i][i - 1] * mu[i - 1]) / denominator
        nu[i] = -matr[i][i + 1] / denominator

    # Последний шаг прогонки
    mu[n - 1] = (vec[n - 1] - matr[n - 1][n - 2] * mu[n - 2]) / (matr[n - 1][n - 1] + matr[n - 1][n - 2] * nu[n - 2])

    # Обратный ход (обратный проход)
    res[n - 1] = mu[n - 1]
    for i in range(n - 2, -1, -1):
        res[i] = mu[i] + nu[i] * res[i + 1]

    return res


def cubic_spline(x, y, a, b):
    n = len(x)
    h = np.diff(x)  # Оставляем h без вставки нуля, это длины отрезков между узлами

    # Проверка на ошибки
    if n < 2:
        return None, None, 1  # Недостаточно точек
    if not np.all(np.diff(x) > 0):
        return None, None, 2  # Нарушен порядок возрастания

    matrix = np.zeros((n, n))  # Трехдиагональная матрица
    F = np.zeros(n)            # Вектор правых частей

    # Условие на c_0 = A
    matrix[0][0] = 1
    F[0] = a

    # Заполнение главной диагонали
    for i in range(1, n - 1):
        matrix[i][i] = 2 * (h[i - 1] + h[i])  # Индексация скорректирована для h

    # Заполнение над главной диагональю
    for i in range(1, n - 1):
        matrix[i][i + 1] = h[i]  # Над главной диагональю идет h[i]

    # Заполнение под главной диагональю
    for i in range(1, n - 1):
        matrix[i][i - 1] = h[i - 1]  # Под главной диагональю идет h[i-1]

    # Условие на B
    matrix[n - 1][n - 2] = 1 / 2
    matrix[n - 1][n - 1] = 1

    # Заполнение вектора правых частей F
    F[n - 1] = (3 / h[n - 2]) * (b - (y[n - 1] - y[n - 2]) / h[n - 2])

    for i in range(1, n - 1):
        F[i] = 6 * ((y[i + 1] - y[i]) / h[i] - (y[i] - y[i - 1]) / h[i - 1])

    # print("Матрица:")
    # print(matrix)
    # print("Вектор F:")
    # print(F)

    return matrix, F, 0



# Чтение данных из файла
def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    x = np.array([float(val) for val in lines[0].split()])
    y = np.array([float(val) for val in lines[1].split()])
    a = float(lines[2].strip())  # Вторая производная в начале
    b = float(lines[3].strip())  # Первая производная в конце
    
    return x, y, a, b

# Запись данных в файл
def write_output(filename, x, y, second_derivatives):
    df = pd.DataFrame({
        'x': x,
        'f(x)': y,
        "f''(x)": second_derivatives,
    })
    df.to_csv(filename, index=False)

# Главная функция, связывающая ввод и вывод
def main(input_file, output_file):
    x, y, a, b = read_input(input_file)

    matr, F, IER = cubic_spline(x, y, a, b)

    if IER == 0:
        second_derivatives = solve_system(matr, F, len(x))
        write_output(output_file, x, y, second_derivatives)
        print(f"Результаты записаны в файл: {output_file}")
    else:
        print(f"Ошибка при построении сплайна. Код ошибки: {IER}")

# Запуск программы с файлами
input_file = 'test2.txt'
output_file = 'output.csv'
main(input_file, output_file)

----
----
----
----
----

---
---
---

In [3]:
import numpy as np
import pandas as pd


def solve_system(lower_diag, main_diag, upper_diag, vec, n):
    # Метод прогонки для решения трехдиагональной системы уравнений
    res = np.zeros(n)
    mu = np.zeros(n)
    nu = np.zeros(n)

    # Прямой ход прогонки
    mu[0] = vec[0] / main_diag[0]
    nu[0] = -upper_diag[0] / main_diag[0]

    for i in range(1, n - 1):
        denominator = main_diag[i] + lower_diag[i - 1] * nu[i - 1]
        mu[i] = (vec[i] - lower_diag[i - 1] * mu[i - 1]) / denominator
        nu[i] = -upper_diag[i] / denominator

    mu[n - 1] = (vec[n - 1] - lower_diag[n - 2] * mu[n - 2]) / (main_diag[n - 1] + lower_diag[n - 2] * nu[n - 2])

    # Обратный ход прогонки
    res[n - 1] = mu[n - 1]
    for i in range(n - 2, -1, -1):
        res[i] = mu[i] + nu[i] * res[i + 1]

    return res

def cubic_spline(x, y, a, b):
    n = len(x)
    h = np.diff(x)

    if n < 2:
        return None, None, None, None, 1
    if not np.all(np.diff(x) > 0):
        return None, None, None, None, 2

    lower_diag = np.zeros(n - 1)
    main_diag = np.zeros(n)
    upper_diag = np.zeros(n - 1)
    F = np.zeros(n)

    # Заполнение граничных условий
    main_diag[0] = 1
    F[0] = a

    # Заполнение диагоналей и вектора F
    for i in range(1, n - 1):
        lower_diag[i - 1] = h[i - 1]
        main_diag[i] = 2 * (h[i - 1] + h[i])
        upper_diag[i] = h[i]

    main_diag[n - 1] = 1
    lower_diag[n - 2] = 1 / 2
    F[n - 1] = (3 / h[n - 2]) * (b - (y[n - 1] - y[n - 2]) / h[n - 2])

    for i in range(1, n - 1):
        F[i] = 6 * ((y[i + 1] - y[i]) / h[i] - (y[i] - y[i - 1]) / h[i - 1])

    return lower_diag, main_diag, upper_diag, F, 0

def read_input(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    x = np.array([float(val) for val in lines[0].split()])
    y = np.array([float(val) for val in lines[1].split()])
    a = float(lines[2].strip())
    b = float(lines[3].strip())
    
    return x, y, a, b

def write_output(filename, x, y, second_derivatives):
    # Формируем DataFrame
    df = pd.DataFrame({
        'x': x,
        'f(x)': y,
        "f''(x)": second_derivatives,
    })
    # Записываем DataFrame в файл с заданным форматом
    df.to_csv(filename, index=False, float_format="%.6f")


def main(input_file, output_file):
    x, y, a, b = read_input(input_file)
    lower_diag, main_diag, upper_diag, F, IER = cubic_spline(x, y, a, b)

    if IER == 0:
        second_derivatives = solve_system(lower_diag, main_diag, upper_diag, F, len(x))
        write_output(output_file, x, y, second_derivatives)
        print(f"Результаты записаны в файл: {output_file}")
    else:
        if IER == 1:
            print("Ошибка: недостаточно точек для построения сплайна.")
        elif IER == 2:
            print("Ошибка: значения x должны быть строго возрастающими.")
        else:
            print(f"Ошибка при построении сплайна. Код ошибки: {IER}")

# Запуск программы с файлами
input_file = 'test1.txt'
output_file = 'output.csv'
main(input_file, output_file)


Результаты записаны в файл: output.csv


---
---
---

----
----
----

---
---
---

# Пример использования
x = np.array([0, 2, 4, 5])
y = np.array([0, 4, 16, 25])
a, b = 2, 10 


# Пример матрицы и вектора правых частей
matr, F, IER = cubic_spline(x, y, a, b)

# Решаем систему методом прогонки
C = solve_system(matr, F, len(x))

print("Решение системы (вторые производные):")
print(C)

# Вывод таблицы
if IER == 0:
    table = pd.DataFrame({
        'x': x,  
        'f(x)': y,  
        "f''(x)": C,  
    })
    print(table.to_string(index=False))
else:
    print("Ошибка:", IER)


# Тест 2: Линейная функция f(x) = 2x + 1
x = np.array([0, 1, 2, 3, 4])
y = np.array([1, 3, 5, 7, 9])
a = 0  # Вторая производная в начале
b = 2  # Первая производная в конце

# Пример матрицы и вектора правых частей
matr, F, IER = cubic_spline(x, y, a, b)

# Решаем систему методом прогонки
C = solve_system(matr, F, len(x))

print("Решение системы (вторые производные) для линейной функции:")
print(C)

# Вывод таблицы
if IER == 0:
    table = pd.DataFrame({
        'x': x,  
        'f(x)': y,  
        "f''(x)": C,  
    })
    print("Линейная функция f(x) = 2x + 1")
    print(table.to_string(index=False))
else:
    print("Ошибка:", IER)


# Тест 1: Константная функция f(x) = 4
x = np.array([0, 1, 2, 3, 4])
y = np.array([4, 4, 4, 4, 4])
a = 0  # Вторая производная в начале
b = 0  # Первая производная в конце

# Пример матрицы и вектора правых частей
matr, F, IER = cubic_spline(x, y, a, b)

# Решаем систему методом прогонки
C = solve_system(matr, F, len(x))

print("Решение системы (вторые производные) для константной функции:")
print(C)

# Вывод таблицы
if IER == 0:
    table = pd.DataFrame({
        'x': x,  
        'f(x)': y,  
        "f''(x)": C,  
    })
    print("Константная функция f(x) = 4")
    print(table.to_string(index=False))
else:
    print("Ошибка:", IER)


# Тест 4: Кубическая функция f(x) = x^3 - 3x^2 + 2x
x = np.array([0, 1, 2, 3, 4])
y = x**3 - 3 * x**2 + 2 * x  # Кубическая функция
a = -6  # Вторая производная в начале
b = 26  # Первая производная в конце

# Пример матрицы и вектора правых частей
matr, F, IER = cubic_spline(x, y, a, b)

# Решаем систему методом прогонки
C = solve_system(matr, F, len(x))

print("Решение системы (вторые производные) для кубической функции:")
print(C)

# Вывод таблицы
if IER == 0:
    table = pd.DataFrame({
        'x': x,  
        'f(x)': y,  
        "f''(x)": C,  
    })
    print("Кубическая функция f(x) = x^3 - 3x^2 + 2x")
    print(table.to_string(index=False))
else:
    print("Ошибка:", IER)
